# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

In [ ]:
#pip install --upgrade langchain

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [6]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.indexes import VectorstoreIndexCreator

In [9]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [10]:
#pip install docarray

In [11]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [28]:
query ="Please list all your shirts which keeps me warm \
in a table in markdown and summarize each one."

**Note**:
- The notebook uses `langchain==0.0.179` and `openai==0.27.7`
- For these library versions, `VectorstoreIndexCreator` uses `text-davinci-003` as the base model, which has been deprecated since 1 January 2024.
- The replacement model, `gpt-3.5-turbo-instruct` will be used instead for the `query`.
- The `response` format might be different than the video because of this replacement model.

In [29]:
llm_replacement_model = OpenAI(temperature=0, 
                               model='gpt-3.5-turbo-instruct')

response = index.query(query, llm = llm_replacement_model)

In [30]:
display(Markdown(response))



| Name | Description |
| --- | --- |
| Classic Plaid Short-Sleeve Shirt | Made from pure European flax, this linen shirt is lightweight and breathable, perfect for warm summer days. It has a relaxed fit through the chest and sleeves, with a slightly slimmer waist. Features a single patch pocket and a shirttail hem. Machine washable. |
| Men's Tropical Plaid Short-Sleeve Shirt | This shirt is rated UPF 50+ for superior sun protection and is made of wrinkle-resistant polyester. It has a traditional fit that is relaxed through the chest, sleeve, and waist. Features front and back cape venting and two front bellows pockets. Machine washable. |
| Double-Layer Base Layer, Crewneck | This thermal shirt has a double-layered design, with a durable knit outer layer and a soft, breathable inner layer. It is made of a blend of cotton, merino wool, and nylon, and is machine washable. Features rib-knit cuffs and non-chafing flatlock seams. Made in Canada. |

Summary:

- Classic Plaid Short-Sleeve Shirt: Lightweight and breathable linen shirt, relaxed fit, machine washable.
- Men's Tropical Plaid Short-Sleeve

## Step By Step

In [31]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path=file)

In [32]:
docs = loader.load()

In [33]:
docs[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

In [34]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [35]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [36]:
print(len(embed))

1536


In [37]:
print(embed[:5])

[-0.02199048176407814, 0.006746508646756411, -0.018174780532717705, -0.03918623551726341, -0.01404528971761465]


In [38]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [40]:
query = "Please suggest a shirt which can keep me warm"

In [41]:
docs = db.similarity_search(query)

In [42]:
len(docs)

4

In [43]:
docs[0]

Document(page_content=": 103\nname: Double-Layer Base Layer, Crewneck\ndescription: When it's cold outside, the first layer is the most important - and the double-layered design of these men's thermal shirts provides the ideal solution. Fabric & Care: Durable knit outer layer combines 50% cotton/ 40% merino wool/ 10% nylon, with an inside layer of soft, breathable, itch-free 100% cotton. Machine wash and dry. Construction: Super-soft, warm two-layer construction won’t bind. Rib-knit cuffs with Lycra elastane hold their shape. Nonchafing flatlock seams for superior comfort. Made in Canada.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 103})

In [49]:
retriever = db.as_retriever()

In [50]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)

In [51]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [52]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts which are warm in winters in a table in markdown and summarize each one.") 


In [53]:
display(Markdown(response))

| Name                                      | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [55]:

qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [56]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [57]:
response = qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [58]:
display(Markdown(response))

| Shirt Name                           | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [59]:
response = index.query(query, llm=llm)

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

Reminder: Download your notebook to you local computer to save your work.

### Explained 4 ways to extraction information from documents for a given quesion
1 - using vectorstoreindex
```python
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])
```
2 - just a similarity search in you documents no llm attached
```python
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)
```

3- using retriever 
``` python
retriever = db.as_retriever()
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)
```
4 - using stuff chain
``` python
retriever = db.as_retriever()
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)
``` 